# The following excercises come from a university homework that we did

## 1. **RPGCharacterKeeper**

Készíts egy osztályt (**RPGCharacterKeeper**), amely szerepjátékokban a játékosok karaktereit és azok különböző képességeihez rendelt módosító pontszámait képes eltárolni. A képességekhez tartozó pontszámok egész számok, lehetnek negatívak is. A példányosított **RPGCharacterKeeper** objektumba bejegyezhetők a karakterek nevei, majd az egyes karakterekhez hozzáadhatók képességek és az azokhoz tartozó módosító pontszámok.

Az osztály neve legyen **RPGCharacterKeeper**. 6 publikus metódusa van:
* ``add_character(name)``: eltárolja a `name` paraméterben átadott string típusú játékos-karakter nevet.
* ``add_stat(name, statname, modifier)``: rögzíti a `name` nevű játékos-karakter `statname` képességéhez a `modifier` pontszámot. Ha a `name` karakter nincs eltárolva az osztályban, `CharacterNotExistsException`-t vált ki.
* ``get_characters_with_long_names()-> [str]``: visszaadja azon játékos-karakterek neveit egy listában, akiknek neve legalább 3, szóközzel elválasztott tagból áll.
* ``get_max_modifiers() -> dict(str, int)``: minden játékos-karakterhez (kulcs) visszaadja az adott karakterhez rendelt legnagyobb módosító pontszámot (érték) egy szótárban. Ha egy képesség módosító pontszáma kisebb, mint 0, adjon vissza 0-t.
* ``get_has_modifier(stat) -> [str]``: Visszaadja azon játékos-karakterek neveinek listáját, akiknek `stat` képességéhez van módosító pontszámuk eltárolva az osztályban.
* ``get_positive_chars() -> [str]``: Visszaadja azon játékos-karakterek neveinek listáját, akikhez több pozitív módosító pontszám van rendelve, mint negatív.

In [3]:
#@title Megoldás

class CharacterNotExistsException(Exception):
        def __init__(self, name):
            super().__init__(name)

class RPGCharacterKeeper:
    def __init__(self):
        self.__character_list = []
        self.__stats = {}

    def add_character(self, name):
        self.__character_list.append(name)

    def add_stat(self, name, statname, modifier):
        if name not in self.__character_list:
            raise CharacterNotExistsException(name)

        else:
            self.__stats[(name, statname)] = modifier

    def get_characters_with_long_names(self):
        return list(filter(lambda x: x!="", [name if name.count(" ") >= 2 else "" for name in self.__character_list]))

    def get_max_modifiers(self):
        res = {}
        for name in self.__character_list:
            res[name] = 0
        for (name, _) in self.__stats:
            if res[name] < self.__stats[(name, _)]:
                res[name] = self.__stats[(name, _)]
        return res

    def get_has_modifier(self, stat):
        res = []
        for name in self.__character_list:
            if (name, stat) in self.__stats:
                res.append(name)
        return res

    def get_positive_chars(self):
        res = {}
        for (name, _) in self.__stats:
            if self.__stats[(name, _)] > 0:
                if name not in res:
                    res[name] = 1
                else:
                    res[name] += 1

            elif self.__stats[(name, _)] < 0:
                if name not in res:
                    res[name] = -1
                else:
                    res[name] -= 1
        return list(filter(lambda x: x!= "",[name if res[name] > 0 else "" for name in res]))


In [2]:
#@title Tesztek

import unittest

class TestRPGCharacterKeeper(unittest.TestCase):
    def setUp(self):
        self.character_keeper = RPGCharacterKeeper()
        self.character_keeper.add_character("Vax'ildan")

    def test_add_character(self):
        with self.assertRaises(CharacterNotExistsException):
            self.character_keeper.add_stat("Percy", "Wisdom", +3)

        try:
            self.character_keeper.add_character("Percy")
            self.character_keeper.add_stat("Percy", "Wisdom", +3)
        except InvalidPlayerException:
            self.fail("raised CharacterNotExistsException, but character should exist")

    def test_get_characters_with_long_names(self):
        long_names = self.character_keeper.get_characters_with_long_names()
        self.assertEqual(long_names, [])

        self.character_keeper.add_character("Percival de Rolo")
        self.character_keeper.add_character("Beauregard")
        long_names = self.character_keeper.get_characters_with_long_names()
        self.assertEqual(long_names, ["Percival de Rolo"])

        self.character_keeper.add_character("Keyleth of the Air Ashari")
        long_names = self.character_keeper.get_characters_with_long_names()
        self.assertEqual(len(long_names), 2)
        self.assertTrue("Percival de Rolo" in long_names)
        self.assertTrue("Keyleth of the Air Ashari" in long_names)

    def test_get_max_modifiers(self):
        max_modifiers = self.character_keeper.get_max_modifiers()
        self.assertEqual(max_modifiers["Vax'ildan"], 0)

        self.character_keeper.add_stat("Vax'ildan", "Strength", -1)
        self.assertEqual(max_modifiers["Vax'ildan"], 0)

        self.character_keeper.add_character("Pike Trickfoot")
        self.character_keeper.add_stat("Pike Trickfoot", "Dexterity", -1)
        self.character_keeper.add_stat("Pike Trickfoot", "Charisma", +2)
        self.character_keeper.add_stat("Pike Trickfoot", "Wisdom", +5)
        max_modifiers = self.character_keeper.get_max_modifiers()
        self.assertEqual(max_modifiers["Vax'ildan"], 0)
        self.assertEqual(max_modifiers["Pike Trickfoot"], 5)

    def test_get_has_modifier(self):
        self.character_keeper.add_stat("Vax'ildan", "Strength", -1)
        has_modifier = self.character_keeper.get_has_modifier("Wisdom")
        self.assertEqual(has_modifier, [])
        has_modifier = self.character_keeper.get_has_modifier("Strength")
        self.assertEqual(has_modifier, ["Vax'ildan"])

        self.character_keeper.add_character("Grog")
        self.character_keeper.add_stat("Grog", "Strength", +6)
        self.character_keeper.add_stat("Grog", "Dexterity", +3)

        self.character_keeper.add_character("Scanlan")
        self.character_keeper.add_stat("Scanlan", "Charisma", +5)

        has_modifier = self.character_keeper.get_has_modifier("Strength")
        self.assertEqual(len(has_modifier), 2)
        self.assertTrue("Vax'ildan" in has_modifier)
        self.assertTrue("Grog" in has_modifier)

    def test_get_positive_chars(self):
        positive_chars = self.character_keeper.get_positive_chars()
        self.assertEqual(positive_chars, [])

        self.character_keeper.add_stat("Vax'ildan", "Strength", -1)
        positive_chars = self.character_keeper.get_positive_chars()
        self.assertEqual(positive_chars, [])

        self.character_keeper.add_stat("Vax'ildan", "Dexterity", +5)
        self.character_keeper.add_stat("Vax'ildan", "Wisdom", +3)
        positive_chars = self.character_keeper.get_positive_chars()
        self.assertEqual(positive_chars, ["Vax'ildan"])

        self.character_keeper.add_character("Vex'ahlia")
        self.character_keeper.add_stat("Vex'ahlia", "Strength", -2)
        self.character_keeper.add_stat("Vex'ahlia", "Charisma", +3)
        positive_chars = self.character_keeper.get_positive_chars()
        self.assertEqual(positive_chars, ["Vax'ildan"])

def suite():
    suite = unittest.TestSuite()
    testfuns = ["test_add_character", "test_get_characters_with_long_names",
                "test_get_max_modifiers", "test_get_has_modifier",
                "test_get_positive_chars"]
    [suite.addTest(TestRPGCharacterKeeper(fun)) for fun in testfuns]
    return suite

runner = unittest.TextTestRunner(verbosity=2)
runner.run(suite())

test_add_character (__main__.TestRPGCharacterKeeper) ... ok
test_get_characters_with_long_names (__main__.TestRPGCharacterKeeper) ... ok
test_get_max_modifiers (__main__.TestRPGCharacterKeeper) ... ok
test_get_has_modifier (__main__.TestRPGCharacterKeeper) ... ok
test_get_positive_chars (__main__.TestRPGCharacterKeeper) ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.022s

OK


<unittest.runner.TextTestResult run=5 errors=0 failures=0>

## 2. **Hamis felfedezési arány** (False discovery rate)

Multiclass klasszifikáció esetén a mintaelemeink címkéjét k kategória egyikébe becsüljük (például kutya, macska, papagáj, stb.). Ha a becsléshez neuronhálót használunk, tipikusan, mintaelemenként egy-egy k elemű, valószínűségeket tartalmazó vektort kapunk, ahol az egyes valószínűségek a mintaelem egyes kategóriákba tartozásának valószínűségét reprezentálják. Ha szeretnénk ezekből a vektorokból megmondani, hogy egy-egy mintaelem melyik kategóriába tartozik a legnagyobb valószínűség szerint, akkor elég megmondani minden egyes vektorban a maximális elem indexét. Így megkapjuk a becsült kategóriákat.

Klasszifikációs modellek teljesítményének mérésére különböző metrikák léteznek (pl. pontosság - accuracy, precizitás - precision, szenzitivitás - recall, stb.). Az egyes metrikák a teljesítményt csak bizonyos szempontok szerint értékelik, jellemzően, egy modell egyetlen metrikával történő kiértékelése nem ad teljes képet a modell klasszifikációs teljesítményéről.

Ebben a feladatban a **hamis felfedezési arány** (False discovery rate) metrikát kell implementálnod a **multiclass** (kettőnél több kategóriás) **klasszifikáció esetére.** A multiclass hamis felfedezési arány, kategóriák felett átlagolva, azt adja meg, hogy az általunk egy adott kategóriába sorolt elemek közt milyen arányban vannak azok az elemek, melyek valódi címkéje szerint nem a becsült kategóriába tartoznak. Minél nagyobb ez az érték, annál rosszabbak a becsléseink.

Számolásához a bináris (két kategóriás) esetből indulhatunk ki. Ilyenkor az egyik, választott kategóriánk a pozitív kategória, míg a másik a negatív kategória. A bináris hamis felfedezési arány így a következő:

$$ FDR = \dfrac{FP}{FP+VP} $$

ahol VP a valódi pozitívok (azaz ahol a pozitív kategóriát helyesen becsültük) száma, FP pedig a fals pozitívok száma (azaz ahol a pozitív kategóriát helytelenül becsültük). A nevező tehát egyenlő azzal, hogy hány elemet becsültünk a pozitív kategóriába.

Multiclass esetben minden kategóriára számoljuk a fenti arányt úgy, hogy az aktuális kategória a pozitív kategória és mindegyik másik kategória együttvéve a negatív kategória. Az így, kategóriánként kapott FDR értékeknek az átlaga (makro-átlagolás) adja meg a multiclass FDR metrikát.

A feladat, hogy implementáld a `false_discovery_rate` függvényt, ami két paramétert kap:
*   `y_pred` tartalmazza becsült valószínűségeket (ez egy (m, k) alakú tömb, ahol  m  a mintaelemek és  k  a kategóriák száma)
*   `y_true` tartalmazza az igazi kategóriacímkéket (ez egy (m,) alakú tömb)

A függvény egy számot ad vissza, a kategóriákra egyenként számolt bináris FDR értékek átlagát.

**Kikötés:**  Az implementációt vektoros módon, NumPy-ban, ciklusok és egyéb, annak megfelelő Python konstrukciók használata nélkül kell elkészíteni. További részletek a notebook végén.

In [4]:
#@title Megoldás

import numpy as np

# Your solution ->

def false_discovery_rate(y_pred, y_true):
    num_samples, num_categories = y_pred.shape

    y_true_binary = np.zeros((num_samples, num_categories))
    y_true_binary[np.arange(num_samples), y_true] = 1       # az i-edik oszlop azt mutatja, hogy az i-edik kategória a pozitív kategória

    max_indexes = np.argmax(y_pred, 1)
    y_pred_binary = np.zeros((num_samples, num_categories))
    y_pred_binary[np.arange(num_samples), max_indexes] = 1

    fp = np.sum((y_pred_binary == 1) & (y_true_binary == 0), axis=0)

    vp = np.sum((y_pred_binary == 1) & (y_true_binary == 1), axis=0)

    fdr = fp / (fp + vp)

    avg_fdr = np.mean(fdr)

    return avg_fdr




In [5]:
#@title Tesztek

import unittest

class TestFDR(unittest.TestCase):

    def test_two_classes(self):
        two_class_preds = np.array([[0.4, 0.6], [0.8, 0.2], [0.55, 0.45], [0.1, 0.9]])  # [1,0,0,1]
        two_class_labels = np.array([0,0,1,1])
        self.assertAlmostEqual(false_discovery_rate(two_class_preds, two_class_labels), 1./2.)

    def test_three_classes(self):
        three_class_preds = np.array([[0.4, 0.3, 0.3], [0.1, 0.5, 0.4],
                                     [0.3, 0.2, 0.5], [0.4, 0.25, 0.35]])  # [0,1,2,0]
        three_class_labels = np.array([0, 1, 2, 2])
        self.assertAlmostEqual(false_discovery_rate(three_class_preds, three_class_labels), 1./6.)

    def test_four_classes(self):
        four_class_preds = np.array([[1., 0., 0., 0.], [1., 0., 0., 0.],
                                     [0., 0., 1., 0.], [0., 0., 1., 0.],
                                     [0., 1., 0., 0.], [0., 0., 0., 1.],
                                     ])  # [0,0,2,2,1,3]
        four_class_labels = np.array([0, 2, 1, 1, 1, 3])
        self.assertAlmostEqual(false_discovery_rate(four_class_preds, four_class_labels), 3./8.)

def suite():
    suite = unittest.TestSuite()
    testfuns = ["test_two_classes", "test_three_classes", "test_four_classes"]
    [suite.addTest(TestFDR(fun)) for fun in testfuns]
    return suite

runner = unittest.TextTestRunner(verbosity=2)
runner.run(suite())

test_two_classes (__main__.TestFDR) ... ok
test_three_classes (__main__.TestFDR) ... ok
test_four_classes (__main__.TestFDR) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.020s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>

## 3. **Sudoku**

Készíts egy **SudokuBoard** nevű osztályt, ami a Sudoku táblát reprezentálja! A Sudoku egy 9x9-es tábla, ami kilenc darab 3x3-as blokkra van osztva a sorok és oszlopok mentén. 1-től 9-ig kell elhelyeznünk a számokat úgy, hogy minden sorban, oszlopban, és blokkban a számok egyszer szerepeljenek.

Emellett az osztály a következő tagfüggvényekkel kell, hogy rendelkezzen:

*   `can_place(row_idx, col_idx, num) -> bool` : A függvény egy logikai értéket ad vissza, mely megadja, hogy a Sudoku szabályai szerint elhelyezhető-e a `num` szám a tábla `col_idx`, `row_idx` pozícióján. A sorok és oszlopok indexelése nullától kezdődik.
*   `place(row_idx, col_idx, num) -> bool` : A függvény működése azonos az előző függvényével, azt leszámítva, hogy ha a megadott szám elhelyezhető a tábla megadott pozícióján, akkor ez a függvény el is helyezi azt.

*   `get_num_empty_cols() -> int`: A függvény visszaadja az üres oszlopok számát.
*   `get_block_pos_with_missing_num(num) -> ndarray(n_blocks_ret, 2)`: A függvény egy (n_blocks_ret, 2) alakú tömbben visszaadja azoknak a 3x3-as blokkoknak a pozícióját, melyekbe még nem került be a `num` szám. A blokkok pozícióját darabonként két index adja meg. Ha mátrixokként tekintünk a táblára, akkor például a bal alsó blokk pozíciója `(2, 0)` lesz.
*   `get_row_idx_with_minimal_max() -> int or None`:  A függvény megadja annak a nemüres sornak az indexét, melyben a legnagyobb szám minimális az egyes nemüres sorokban elhelyezett legnagyobb számokat tekintve. Ha több ilyen van, az egyiket kell visszadni közülük. Ha minden sor üres, a függvény `None` értéket adjon vissza!
*   `get_num_cols_with_two_nums_present(num1, num2) -> int` : A függvény visszaadja azoknak az oszlopoknak a számát, ahol a `num1` és `num2` számok egyidejűleg jelen vannak. Feltételezhető, hogy a függvény két különböző számot kap argumentumként.

**Kikötés:**  Az implementációt vektoros módon, NumPy-ban, ciklusok és egyéb, annak megfelelő Python konstrukciók használata nélkül kell elkészíteni. További részletek a notebook végén.



In [8]:
#@title Megoldás

import numpy as np

# Your solution ->

class SudokuBoard:
    def __init__(self) -> None:
        self.__table = np.zeros((9,9),dtype=np.int8)

    def can_place(self, row_idx, col_idx, num) -> bool:
        block_start_row = row_idx // 3
        block_start_column = col_idx // 3
        if self.__table[row_idx, col_idx] != 0:
            return False
        elif num in self.__table[row_idx, :]:
            return False
        elif num in self.__table[:, col_idx]:
            return False
        elif num in self.__table[block_start_row * 3 : block_start_row * 3 + 3, block_start_column * 3 : block_start_column * 3 + 3]:
            return False
        else:
            return True

    def place(self, row_idx, col_idx, num) -> bool:
        if not self.can_place(row_idx, col_idx, num):
            return False
        else:
            self.__table[row_idx, col_idx] = num
            return True

    def get_num_empty_cols(self) -> int:
        return 9 - np.count_nonzero(np.count_nonzero(self.__table, axis=0))

    def get_num_cols_with_two_nums_present(self, num1, num2) -> int:
        num_one = self.__table == num1
        num_one_in_cols = np.any(num_one, axis=0)
        num_two = self.__table == num2
        num_two_in_cols = np.any(num_two, axis=0)
        return np.sum(num_one_in_cols & num_two_in_cols)

    def get_row_idx_with_minimal_max(self) -> int or None:
        max_vals = np.max(self.__table, axis=1)
        a = np.array(max_vals)
        a[a==0] = 100
        if(a[np.argmin(a)] != 100):
            return np.argmin(a)
        else:
            return None

    def get_block_pos_with_missing_num(self, num):
        num_in_block_cols = (np.where(self.__table == num)[0]) // 3
        num_in_block_rows = (np.where(self.__table == num)[1]) // 3
        blocks = np.zeros((3,3))
        blocks[num_in_block_cols, num_in_block_rows] = 1    # oda rakunk 1-est, ahol benne van a num
        result = np.argwhere(blocks == 0)
        return result

In [7]:
#@title Tesztek

import unittest

class TestSudoku(unittest.TestCase):
    def test_can_place(self):
        sb = SudokuBoard()
        self.assertTrue(sb.can_place(0, 0, 5))
        self.assertTrue(sb.can_place(8, 0, 5))
        self.assertTrue(sb.can_place(0, 8, 5))
        self.assertTrue(sb.can_place(4, 6, 5))
        self.assertTrue(sb.can_place(8, 8, 5))

        sb.place(3, 3, 5)
        self.assertFalse(sb.can_place(3, 3, 7))
        self.assertFalse(sb.can_place(5, 4, 5))
        self.assertFalse(sb.can_place(7, 3, 5))
        self.assertFalse(sb.can_place(3, 7, 5))

        self.assertTrue(sb.can_place(2, 4, 5))
        self.assertTrue(sb.can_place(3, 4, 6))

    def test_place(self):
        sb = SudokuBoard()
        self.assertTrue(sb.place(2, 7, 4))
        self.assertFalse(sb.place(2, 7, 4))
        self.assertFalse(sb.place(2, 7, 5))
        self.assertTrue(sb.place(2, 0, 5))
        self.assertFalse(sb.place(2, 8, 5))

    def test_get_num_empty_cols(self):
        sb = SudokuBoard()
        self.assertEqual(sb.get_num_empty_cols(), 9)
        sb.place(3, 3, 9)
        self.assertEqual(sb.get_num_empty_cols(), 8)

        to_insert = [(1, 0, 3), (2, 7, 3), (2, 1, 4)]
        [sb.place(*t) for t in to_insert]
        self.assertEqual(sb.get_num_empty_cols(), 5)
        to_insert = [(5, 8, 4), (8, 0, 4), (3, 7, 8)]
        [sb.place(*t) for t in to_insert]
        self.assertEqual(sb.get_num_empty_cols(), 4)

    def test_get_block_pos_with_missing_num(self):
        sb = SudokuBoard()
        ret = sb.get_block_pos_with_missing_num(1)
        self.assertEqual(ret.shape, (9, 2))
        self.assertGreaterEqual(np.amin(ret), 0)
        self.assertLessEqual(np.amax(ret), 2)
        ret_flat_idxs = np.ravel_multi_index((ret[:,0], ret[:,1]), dims=(3,3))
        self.assertEqual(tuple(np.sort(ret_flat_idxs)), tuple(range(9)))

        to_insert = [(1, 0, 3), (2, 7, 3), (2, 1, 4), (5, 8, 4), (8, 0, 4), (3, 7, 8)]
        [sb.place(*t) for t in to_insert]
        ret = sb.get_block_pos_with_missing_num(1)
        self.assertEqual(ret.shape, (9, 2))
        self.assertGreaterEqual(np.amin(ret), 0)
        self.assertLessEqual(np.amax(ret), 2)
        ret_flat_idxs = np.ravel_multi_index((ret[:,0], ret[:,1]), dims=(3,3))
        self.assertEqual(tuple(np.sort(ret_flat_idxs)), tuple(range(9)))
        ret = sb.get_block_pos_with_missing_num(4)
        self.assertEqual(ret.shape, (6, 2))
        self.assertGreaterEqual(np.amin(ret), 0)
        self.assertLessEqual(np.amax(ret), 2)
        ret_flat_idxs = np.ravel_multi_index((ret[:,0], ret[:,1]), dims=(3,3))
        self.assertEqual(tuple(np.sort(ret_flat_idxs)), (1,2,3,4,7,8))

        to_insert = [(4, 2, 4), (1, 4, 4), (3, 3, 4), (0, 6, 4), (6, 7, 4)]
        [sb.place(*t) for t in to_insert]
        ret = sb.get_block_pos_with_missing_num(4)
        self.assertEqual(ret.shape, (1, 2))
        self.assertGreaterEqual(np.amin(ret), 0)
        self.assertLessEqual(np.amax(ret), 2)
        ret_flat_idxs = np.ravel_multi_index((ret[:,0], ret[:,1]), dims=(3,3))
        self.assertEqual(tuple(ret_flat_idxs), (7,))

        sb.place(7, 5, 4)
        ret = sb.get_block_pos_with_missing_num(4)
        self.assertEqual(ret.shape, (0, 2))

    def test_get_row_idx_with_minimal_max(self):
        sb = SudokuBoard()
        self.assertEqual(sb.get_row_idx_with_minimal_max(), None)
        sb.place(3, 3, 9)
        self.assertEqual(sb.get_row_idx_with_minimal_max(), 3)

        to_insert = [(1, 0, 3), (2, 7, 3)]
        [sb.place(*t) for t in to_insert]
        self.assertIn(sb.get_row_idx_with_minimal_max(), [1, 2])

        sb.place(2, 1, 4)
        self.assertEqual(sb.get_row_idx_with_minimal_max(), 1)

        to_insert = [(3, 8, 1), (8, 0, 2), (3, 7, 8)]
        [sb.place(*t) for t in to_insert]
        self.assertEqual(sb.get_row_idx_with_minimal_max(), 8)

    def test_get_num_cols_with_two_nums_present(self):
        sb = SudokuBoard()
        self.assertEqual(sb.get_num_cols_with_two_nums_present(5, 7), 0)
        sb.place(3, 3, 9)
        self.assertEqual(sb.get_num_cols_with_two_nums_present(9, 1), 0)
        self.assertEqual(sb.get_num_cols_with_two_nums_present(9, 8), 0)
        sb.place(5, 3, 5)
        self.assertEqual(sb.get_num_cols_with_two_nums_present(9, 5), 1)

        to_insert = [(3, 8, 5), (5, 2, 9), (6, 2, 5), (6, 3, 1)]
        [sb.place(*t) for t in to_insert]
        self.assertEqual(sb.get_num_cols_with_two_nums_present(5, 9), 2)
        self.assertEqual(sb.get_num_cols_with_two_nums_present(9, 5), 2)
        self.assertEqual(sb.get_num_cols_with_two_nums_present(9, 1), 1)

        to_insert = [(6, 7, 9), (7, 4, 9), (0, 2, 2), (0, 7, 5), (5, 0, 3)]
        [sb.place(*t) for t in to_insert]
        self.assertEqual(sb.get_num_cols_with_two_nums_present(5, 9), 3)
        self.assertEqual(sb.get_num_cols_with_two_nums_present(9, 3), 0)

def suite():
    suite = unittest.TestSuite()
    testfuns = ["test_can_place", "test_place", "test_get_num_empty_cols",
                "test_get_block_pos_with_missing_num", "test_get_row_idx_with_minimal_max",
                "test_get_num_cols_with_two_nums_present"]
    [suite.addTest(TestSudoku(fun)) for fun in testfuns]
    return suite

runner = unittest.TextTestRunner(verbosity=2)
runner.run(suite())

test_can_place (__main__.TestSudoku) ... ok
test_place (__main__.TestSudoku) ... ok
test_get_num_empty_cols (__main__.TestSudoku) ... ok
test_get_block_pos_with_missing_num (__main__.TestSudoku) ... ok
test_get_row_idx_with_minimal_max (__main__.TestSudoku) ... ok
test_get_num_cols_with_two_nums_present (__main__.TestSudoku) ... ok

----------------------------------------------------------------------
Ran 6 tests in 0.029s

OK


<unittest.runner.TextTestResult run=6 errors=0 failures=0>